1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> vector DB 에 저장
4. 질문이 있을 때, vector db 에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax_with_markdown.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

225

In [98]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3.2")

In [105]:
import os
# from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# load_dotenv()

index_name = "tax-markdown-index"

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


# Ollama embedding을 사용하여 Pinecone에 문서 저장할때 ( message length too large 에러 방지 )
# 처음 database를 생성하는 부분
database = PineconeVectorStore.from_documents(document_list, embedding=embeddings, index_name=index_name)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 17 Apr 2025 06:26:09 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '5173', 'x-pinecone-request-id': '4747342347664566211', 'x-envoy-upstream-service-time': '0', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4195820 bytes, the limit is: 4194304 bytes","details":[]}


In [ ]:
query = "제 55조 기준으로 연봉 5천만원인 직장인의 소득세는 얼마인가요?"

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2")

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/Users/nhn/Desktop/llm-application/lib/python3.10/site-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
from langchain.chains import RetrievalQA

retriever = database.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever = database.as_retriever(), 
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
ai_message =qa_chain({"query": query})

In [ ]:
ai_message

{'query': '제 55조 기준으로 연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '<고용주고용노동법 제 3 조 (고용주고용노동의 순서)> \n① 고용주와 고용노동이 그間에生기得 있는 주거비(당일 사용) 및 기타 경비는 주택임대소득에 포함한다. \n\n② 고용주가 고용노동을 해지한 때는 고용노동이 일시적으로 생기게 되는 주거비 및 기타 경비를 포함한 소득은 주택임대소득에 포함한다. \n\n③ 고용주와 고용노동이 그間에 생기得 있는 기타 소득은 고용주고용노동법 제 21 조의 규정에 따라 계산할 수 있다.'}